In [1]:
dataset_name = "cifar10"
model_type = "mlp"
nump_epochs = 40
pth = "./trajectories"

In [2]:
!pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

print(torch.cuda.is_available())


img_size = 32
ch_size = 3

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(img_size*img_size*ch_size, 16)  # First fully connected layer
        self.fc4 = nn.Linear(16, 10)      # Output layer

    def forward(self, x):
        x = x.view(-1, img_size*img_size*ch_size)  # Flatten the input
        x = F.relu(self.fc1(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)
    



True


In [4]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# Instantiate early stopping
early_stopping = EarlyStopping(patience=3, min_delta=0.01)

In [5]:
if dataset_name == "mnist":
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)
else:
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    transform2 = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)                              
    test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform2)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)


# Define the training loop
def train(epoch, model, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.cuda())
        loss = F.nll_loss(output, target.cuda())
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Define the test loop
def test(epoch, model, optimizer):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data.cuda())
            test_loss += F.nll_loss(output, target.cuda(), reduction='sum').item()  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.cuda().view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')
    return test_loss


Files already downloaded and verified


In [6]:
import os


# Initialize the model and optimizer
model = MLP()
model=model.cuda()
optimizer = optim.SGD(model.parameters(), lr= 0.001, momentum=0.9, weight_decay=1e-4)
    
    
def save_model(epoch, model):
    # Save the model after each epoch
    save_path = f'{pth}/models_{model_type}_{dataset_name}_lowerlr/model_epoch_{epoch}.pt'
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    torch.save(model.state_dict(), save_path)
    print(f'Model saved to {save_path}')
    
# Train and test the model with early stopping
save_model(0, model)
for epoch in range(1, nump_epochs + 1):
    train(epoch)
    val_loss = test()
    early_stopping(val_loss)    
    save_model(epoch, model)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break
    



Model saved to /home/elhamod/projects/AE/trajectories/models_mlp_cifar10_lowerlr/model_epoch_0.pt
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.328299
Train Epoch: 1 [3200/50000 (6%)]	Loss: 2.171441
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.152892
Train Epoch: 1 [9600/50000 (19%)]	Loss: 2.043013
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.918759
Train Epoch: 1 [16000/50000 (32%)]	Loss: 2.000021
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.892191
Train Epoch: 1 [22400/50000 (45%)]	Loss: 1.835348
Train Epoch: 1 [25600/50000 (51%)]	Loss: 2.195186
Train Epoch: 1 [28800/50000 (58%)]	Loss: 1.860140
Train Epoch: 1 [32000/50000 (64%)]	Loss: 1.892031
Train Epoch: 1 [35200/50000 (70%)]	Loss: 1.427150
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.555125
Train Epoch: 1 [41600/50000 (83%)]	Loss: 1.709248
Train Epoch: 1 [44800/50000 (90%)]	Loss: 1.851730
Train Epoch: 1 [48000/50000 (96%)]	Loss: 2.199516

Test set: Average loss: 1.7582, Accuracy: 3880/10000 (39%)

Model saved to /home/elhamod/projects/AE/trajector

Train Epoch: 9 [35200/50000 (70%)]	Loss: 1.662389
Train Epoch: 9 [38400/50000 (77%)]	Loss: 1.563021
Train Epoch: 9 [41600/50000 (83%)]	Loss: 1.720317
Train Epoch: 9 [44800/50000 (90%)]	Loss: 1.600710
Train Epoch: 9 [48000/50000 (96%)]	Loss: 1.376974

Test set: Average loss: 1.5635, Accuracy: 4545/10000 (45%)

Model saved to /home/elhamod/projects/AE/trajectories/models_mlp_cifar10_lowerlr/model_epoch_9.pt
Train Epoch: 10 [0/50000 (0%)]	Loss: 1.560971
Train Epoch: 10 [3200/50000 (6%)]	Loss: 1.871075
Train Epoch: 10 [6400/50000 (13%)]	Loss: 1.597564
Train Epoch: 10 [9600/50000 (19%)]	Loss: 1.609729
Train Epoch: 10 [12800/50000 (26%)]	Loss: 1.483665
Train Epoch: 10 [16000/50000 (32%)]	Loss: 1.525973
Train Epoch: 10 [19200/50000 (38%)]	Loss: 1.620732
Train Epoch: 10 [22400/50000 (45%)]	Loss: 1.257730
Train Epoch: 10 [25600/50000 (51%)]	Loss: 1.874300
Train Epoch: 10 [28800/50000 (58%)]	Loss: 1.447232
Train Epoch: 10 [32000/50000 (64%)]	Loss: 1.483761
Train Epoch: 10 [35200/50000 (70%)]	Los

In [7]:
from torchsummary import summary

img_dim  =32
summary(model, input_size=(3, img_dim, img_dim))  # Input size should match the input dimensions of your model


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 16]          49,168
            Linear-2                   [-1, 10]             170
Total params: 49,338
Trainable params: 49,338
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.00
Params size (MB): 0.19
Estimated Total Size (MB): 0.20
----------------------------------------------------------------
